## Backpropagation 

Backpropagation is a way to training neural networks, where we know the derivatives of each layer. It uses gradient descent as covered in the previous notebook. 

We'll no longer consider just a single input (a rather unrealistic assumption), but we will consider something fairly simple/restricted: all possible logic (boolean) functions in 4 dimensions.

Let's assume we have 4 inputs and a single output. We want to learn for any given input what the output should be.

Rather than work with boolean functions, we will create vector/matrix representations.

In [1]:
bfn_one(a1::Bool, a2::Bool, a3::Bool, a4::Bool) = a1 | a2 & (~a3 | a4)

bfn_one (generic function with 1 method)

In [2]:
bfn_one(true, false, false, true)

true

In [3]:
bfn_one(false, false, false, true)

false

Okay, but let's generate inputs and outputs in number form (0s and 1s) so we can apply our previous techniques.

In [4]:
inputs = reverse.(Iterators.product(fill(0:1,4)...))[:]

16-element Array{NTuple{4,Int64},1}:
 (0, 0, 0, 0)
 (0, 0, 0, 1)
 (0, 0, 1, 0)
 (0, 0, 1, 1)
 (0, 1, 0, 0)
 (0, 1, 0, 1)
 (0, 1, 1, 0)
 (0, 1, 1, 1)
 (1, 0, 0, 0)
 (1, 0, 0, 1)
 (1, 0, 1, 0)
 (1, 0, 1, 1)
 (1, 1, 0, 0)
 (1, 1, 0, 1)
 (1, 1, 1, 0)
 (1, 1, 1, 1)

The code might be a bit magical if you aren't familiar with Julia, but let's not get sidetracked... it generates all possible inputs to our function. (Yes, of course I Googled how to do it!)

In [5]:
# map single input to Booleans
map(Bool, inputs[1])

# map all inputs to Booleans
inputs_b = map(ip -> map(Bool, ip), inputs)

# apply one example
bfn_one(map(Bool, inputs[1])...)

#apply all
outputs_b = map(ib -> bfn_one(ib...), inputs_b)

outputs = map(Int, outputs_b)

(inputs_b, outputs_b, outputs)

(NTuple{4,Bool}[(false, false, false, false), (false, false, false, true), (false, false, true, false), (false, false, true, true), (false, true, false, false), (false, true, false, true), (false, true, true, false), (false, true, true, true), (true, false, false, false), (true, false, false, true), (true, false, true, false), (true, false, true, true), (true, true, false, false), (true, true, false, true), (true, true, true, false), (true, true, true, true)], Bool[false, false, false, false, true, true, false, true, true, true, true, true, true, true, true, true], [0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1])

Actually we want floating point inputs and outputs (integer makes the zero-ish case unambiguous). Julia allows you to `convert` between types nicely. We also use `collect` to convert a tuple to an array.

In [6]:
inputs_f = map(x -> convert(Array{Float32}, collect(x)), inputs)
outputs_f = convert(Array{Float32}, outputs)

inputs_f, outputs_f

(Array{Float32,1}[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 1.0], [0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 1.0], [0.0, 1.0, 1.0, 0.0], [0.0, 1.0, 1.0, 1.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 1.0], [1.0, 0.0, 1.0, 0.0], [1.0, 0.0, 1.0, 1.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 1.0], [1.0, 1.0, 1.0, 0.0], [1.0, 1.0, 1.0, 1.0]], Float32[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

Great. We can get started properly. We are going to pretend we don't know what this function is (and you can try swapping it out for something else and running the below code).

To start with let's try to learn this with a single layer network as before. We'll then look at multiple layers, and see backpropagation in action.

Now we are going to learn from the entire dataset, rather than just a single input and output as before. To do this we will simply go through all of the input + output pairs in turn, then repeat

In [7]:
# selecting rows
inputs_f[1], outputs_f[1]

(Float32[0.0, 0.0, 0.0, 0.0], 0.0f0)

In [8]:
using LinearAlgebra

In [9]:
# try different starting points
weights = convert(Array{Float32}, [0.2, -0.4, 0.1, -0.2])

# try different alphas
alpha = 0.05

for i = 1:25
    err = 0
    for j = 1:16
        ipt = inputs_f[j]
        opt = outputs_f[j]
        
        prediction = dot(ipt, weights)
        
        error = (prediction - opt) ^ 2
        err += error
        
        delta = prediction - opt
        weights = weights - (alpha * delta * ipt)
    end
    print("Total error: $(err), weights: $(weights)\n")
end

Total error: 9.363550346863724, weights: [0.453436, -0.063565, 0.263041, 0.0728878]
Total error: 3.490099164211415, weights: [0.548826, 0.089625, 0.269098, 0.163558]
Total error: 2.41257899721045, weights: [0.588704, 0.173352, 0.231295, 0.192702]
Total error: 2.034853159724225, weights: [0.608238, 0.227581, 0.186219, 0.201224]
Total error: 1.8283416266677444, weights: [0.619661, 0.266979, 0.144831, 0.202966]
Total error: 1.6982437379668867, weights: [0.627366, 0.297423, 0.109676, 0.202591]
Total error: 1.6131366577657529, weights: [0.633038, 0.321639, 0.0806658, 0.201654]
Total error: 1.5566837736061288, weights: [0.637402, 0.341147, 0.0570024, 0.200657]
Total error: 1.5189289996923294, weights: [0.64083, 0.356948, 0.0377918, 0.199751]
Total error: 1.4935104605599894, weights: [0.643545, 0.369775, 0.0222271, 0.198968]
Total error: 1.4762879777828486, weights: [0.645704, 0.380199, 0.00962687, 0.198307]
Total error: 1.4645411634395922, weights: [0.647424, 0.388671, -0.00056976, 0.197752]

So we have a learning process that at least reduces error. The final results don't seem amazing at first, but actually if we round to the nearest value they are right in all but one case.

In [10]:
for j = 1:16
    ipt = inputs_f[j]
    opt = outputs_f[j]
    prediction = dot(ipt, weights)
    print("Predict: $(prediction), rounded prediction: $(round(prediction)), real value: $(opt)\n")  
end

Predict: 0.0, rounded prediction: 0.0, real value: 0.0
Predict: 0.19525525950307085, rounded prediction: 0.0, real value: 0.0
Predict: -0.04101851377707512, rounded prediction: -0.0, real value: 0.0
Predict: 0.15423674572599572, rounded prediction: 0.0, real value: 0.0
Predict: 0.42301324600363194, rounded prediction: 0.0, real value: 1.0
Predict: 0.6182685055067028, rounded prediction: 1.0, real value: 1.0
Predict: 0.38199473222655683, rounded prediction: 0.0, real value: 0.0
Predict: 0.5772499917296277, rounded prediction: 1.0, real value: 1.0
Predict: 0.6537850102199332, rounded prediction: 1.0, real value: 1.0
Predict: 0.8490402697230041, rounded prediction: 1.0, real value: 1.0
Predict: 0.6127664964428581, rounded prediction: 1.0, real value: 1.0
Predict: 0.808021755945929, rounded prediction: 1.0, real value: 1.0
Predict: 1.0767982562235652, rounded prediction: 1.0, real value: 1.0
Predict: 1.272053515726636, rounded prediction: 1.0, real value: 1.0
Predict: 1.03577974244649, rou

Okay, let's try something more sophisticated, with multiple layers. We are going to try and dive straight into the code, then figure out what is going on. First we change the activation function to a [relu](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)) function. Previously we just had simplistic linear weightings, which won't work for multiple layers.

In [17]:
relu(x) = x > 0 ? x : 0

relu(-0.3), relu(0.3)

(0, 0.3)

We also define the derivative

In [18]:
drelu(x) = x > 0 ? 1 : 0

drelu (generic function with 1 method)

We can generate random matrices quite simply in Julia.

In [19]:
convert(Array{Float32}, rand(4,4))

4×4 Array{Float32,2}:
 0.0685887  0.307153   0.962565  0.504352 
 0.0038114  0.149178   0.604692  0.61558  
 0.348633   0.0675927  0.969178  0.903942 
 0.796645   0.110765   0.349788  0.0240072

Now we actually have everything we need

In [20]:
map(relu, transpose(inputs_f[1]) * weights_1)

1×4 Transpose{Int64,Array{Int64,1}}:
 0  0  0  0

In [21]:
alpha = 0.1

# we are going to add a new layer
hiddenSize = 4

# we want values between -1 and 1
weights_1 = 2 * rand(4, hiddenSize) .- 1
weights_2 = 2 * rand(hiddenSize, 1) .- 1

for i = 1:250
    layer_3_err = 0
    for j in 1:16
        layer_1 = transpose(inputs_f[j])
        layer_2 = map(relu, layer_1 * weights_1)
        layer_3 = layer_2 * weights_2        
        layer_3_err += (layer_3[1] - outputs_f[j]) .^ 2
        layer_3_delta = outputs_f[j] .- layer_3     
        layer_2_delta = layer_3_delta * transpose(weights_2) .* map(drelu, layer_2)
        weights_2 += alpha * transpose(layer_2) * layer_3_delta
        weights_1 += alpha * transpose(layer_1) * layer_2_delta
    end    
    if i % 10 == 0
        println("Error: $(layer_3_err)")
    end
end

Error: 2.1956791744951136
Error: 1.5184971660158275
Error: 1.4018684062932023
Error: 1.1563765987532375
Error: 0.8659545931347518
Error: 0.7286166722972915
Error: 0.6477469635292372
Error: 0.6036690404020246
Error: 0.5867428180375243
Error: 0.5641489610331856
Error: 0.5619398118862209
Error: 0.5549736519180126
Error: 0.5491009120140563
Error: 0.543861047554249
Error: 0.5391048965141519
Error: 0.5347594621618874
Error: 0.5307696377067468
Error: 0.5270871272985168
Error: 0.4991148485611913
Error: 0.4818549619835947
Error: 0.4714584271779233
Error: 0.4646841337965765
Error: 0.46054234531287686
Error: 0.4583334346440867
Error: 0.3457354427114695


In [22]:
for i = 1:16
    layer_1 = transpose(inputs_f[i])
    layer_2 = map(relu, layer_1 * weights_1)
    layer_3 = layer_2 * weights_2
    
    prediction = layer_3[1]
    actual = outputs_f[i]
    
    print("Predict: $(prediction), rounded prediction: $(round(prediction)), real value: $(actual)\n")  
end

Predict: 0.0, rounded prediction: 0.0, real value: 0.0
Predict: 0.05335071183952486, rounded prediction: 0.0, real value: 0.0
Predict: -0.2974743022570351, rounded prediction: -0.0, real value: 0.0
Predict: 0.029845521316776615, rounded prediction: 0.0, real value: 0.0
Predict: 0.6779849496296484, rounded prediction: 1.0, real value: 1.0
Predict: 0.9797686024483634, rounded prediction: 1.0, real value: 1.0
Predict: 0.21696932267318514, rounded prediction: 0.0, real value: 0.0
Predict: 0.9038419079872269, rounded prediction: 1.0, real value: 1.0
Predict: 0.9645602870155497, rounded prediction: 1.0, real value: 1.0
Predict: 0.951680820768274, rounded prediction: 1.0, real value: 1.0
Predict: 1.0521749172538413, rounded prediction: 1.0, real value: 1.0
Predict: 0.9944058083323263, rounded prediction: 1.0, real value: 1.0
Predict: 0.9972941009265819, rounded prediction: 1.0, real value: 1.0
Predict: 0.9928757038445919, rounded prediction: 1.0, real value: 1.0
Predict: 0.994140904359529, ro


So if we round we are actually right 100% of the time (for this example/learning etc). But how did that work. Let's go through it line by line.

`for j in 1:16`

Go though each of our 16 examples. With a real dataset we'd probably use batches of samples.

`layer_1 = transpose(inputs_f[j])`

The first layer is the inputs.

`layer_2 = map(relu, layer_1 * weights_1)`

The second layer performs `relu(x) = x > 0 ? x : 0` on the first layer multiplied by weightings.

`layer_3 = layer_2 * weights_2`        

The third layer (the output) is just a linear weighting of the previous output.

`layer_3_err += (layer_3[1] - outputs_f[j]) .^ 2`

The error is the mean squared mismatch.

`layer_3_delta = outputs_f[j] .- layer_3`

We weight our overall adjustment by the error in the output. 

`layer_2_delta = layer_3_delta * transpose(weights_2) .* map(drelu, layer_2)`

Now we work out the adjustment to the first set of weightings by taking the derivated of the second layer operation mutliplied by the weightings.

`weights_2 += alpha * transpose(layer_2) * layer_3_delta`

We adjust the second layer. Note the alpha term which controls speed of adjustment.

`weights_1 += alpha * transpose(layer_1) * layer_2_delta`

And first layer.

But all of this happens for all of the data. We aren't just updating with a single input/output. We try and move in right overall direction.

The key point is we work backwards adjusting the weights towards better values that reduce error in the output (or adjust in the right direction for the next layer).

## Next Steps

We'll look at how we do this kind of thing for real in the next section. While looking at the low level details are useful both for learning Julia and understanding the core principles it would be extremely tedious and error prone day to day.

We'll use [Flux](https://github.com/FluxML/Flux.jl), a Julia framework for machine learning. It is pure Julia yet can benefit from GPU accleration. It allows us to build and train models in a remarkably concise and clear way.